# CR 2058$\rightarrow$ *2007-06-21 to 2007-07-18*

Authors: Opal Issan

Last modified: Jan 18th, 2022

In [2]:
import numpy as np
import datetime as dt
from scipy import ndimage
import astropy.units as u
import time
from sunpy.coordinates import frames
from sunpy.coordinates import HeliographicCarrington
from astropy.coordinates import SkyCoord
from sunpy.coordinates.sun import carrington_rotation_time
from heliopy.data import ace
from heliopy import spice
from HUX.hux_propagation import apply_hux_f_model
import matplotlib
import matplotlib.pyplot as plt
import astropy.constants as const
import sunpy.map
import os
import scipy
import pfsspy
import astropy
from pfsspy import coords, tracing, fieldline
from model_chain import get_ace_date, pfss2flines, distance_to_coronal_hole_boundary
import matplotlib.dates as mdates

plt.rcParams['savefig.facecolor'] = 'white'
font = {'family': 'serif',
        'size': 12}

matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=12)
matplotlib.rc('ytick', labelsize=12)

ModuleNotFoundError: No module named 'HUX'

# Set time period

In [ ]:
CR = "2058"
start_time = carrington_rotation_time(int(CR)).to_datetime()
end_time = carrington_rotation_time(int(CR)+1).to_datetime()
print(start_time)
print(end_time)

# ACE Trajectory and Observations

In [ ]:
ACE_longitude, ACE_latitude, ACE_r, ACE_vr, ACE_obstime = get_ace_date(start_time=start_time, end_time=end_time)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
ax.plot(ACE_obstime.value, ACE_vr, c="k", ls="--")
ax.set_ylim(250, 900)
ax.set_ylabel("Solar Wind Velocity [km/s]")
ax.set_title("ACE/SWEPAM Observations CR" + str(CR))
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=100))
ax.xaxis.set_major_formatter(mdates.DateFormatter('% Y-% m-% d'))

fig.autofmt_xdate()

# GONG Magnetograms

## Read data from NSO website

In [ ]:
gong_map = sunpy.map.Map('GONG/CR' + str(CR) + '/mrmqs070704t1843c2058_000.fits.gz')
gong_map.meta["bunit"] = "gauss"
gong_map.meta["DATE"] = str(ACE_obstime[-1])
gong_map.meta["DATE_OBS"] = str(ACE_obstime[-1])
gong_map

# PFSS

The PFSS model [(Wang et al. 1992)](https://ui.adsabs.harvard.edu/abs/1992ApJ...392..310W/abstract) solves for the magnetic field $\mathbf{B} = [B_{r}(r, \theta, \phi), B_{\theta}(r, \theta, \phi), B_{\phi}(r, \theta, \phi)]^{\top} \in \mathbb{R}^{3}$, where $\theta \in [0, \pi]$ is Carrington latitude, $\phi \in [0, 2\pi]$ is Carrington longitude, and $r \in [1 R_{S}, r_{\text{PFSS}} ] $ is the radial distance from the center of the Sun. PFSS is an elliptic boundary value problem, satisfying
\begin{equation}\label{curl-b-zero}
   \nabla \times \mathbf{B} = 0
\end{equation}
and 
\begin{equation}\label{div-b-zero}
    \nabla \cdot \mathbf{B} =0.
\end{equation}
We can describe the magnetic field by its potential,
\begin{equation}\label{potential-def}
    \mathbf{B} = \nabla \Psi.
\end{equation}
Then, we obtain the laplace equation
\begin{equation}
    \nabla^2 \Psi = 0, 
\end{equation}
subject to 
\begin{align}
    \frac{\partial \Psi}{\partial r}(r=1, \theta, \phi) &= g(\theta, \phi),\\
    \frac{\partial \Psi}{\partial \theta} (r = r_{\text{PFSS}}, \theta, \phi) &= \frac{\partial \Psi}{\partial \phi} (r=r_{\text{PFSS}}, \theta, \phi) = 0,\\
    \Psi(r= r_{\text{PFSS}}, \theta, \phi) &= 0,\\
    \Psi(r, \theta, \phi=0) &= \Psi(r, \theta, \phi=2\pi),\\
    \Psi(r, \theta=\{0, \pi\}, \phi) &= \frac{1}{2\pi} \int_{\phi=0}^{\phi=2\pi} \Psi(r, \theta=\{0 \pm \epsilon, \pi \pm \epsilon\}, \phi) \text{d} \phi,
\end{align}
where $g(\theta, \phi)$ is set to the GONG data. The PFSS model can be solved via finite-difference numerical schemes or alternatively by spherical harmonic expansion. The upper boundary is usually set to $r_{\text{PFSS}} = 2.5 R_{S}$ to best match observations. 

In [ ]:
# pfss parameters + simulate
pfss_in = pfsspy.Input(br=gong_map, nr=100, rss=2.5)
pfss_out = pfsspy.pfss(pfss_in)

In [ ]:
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(10, 3.5))
pos = ax[0].imshow(gong_map.data, extent=[0,360,-90, 90], origin="lower", aspect="auto", cmap="seismic", vmin=-30, vmax=30)

ax[0].set_xticks([0, 90, 180, 270, 360])
ax[0].set_yticks([-90, -45, 0, 45, 90])
cbar = fig.colorbar(pos, ax=ax[0], orientation="vertical")
cbar.ax.set_ylabel(r'$B_{r}(r=1R_{S})$ [Gauss]', rotation=90)
ax[0].set_xlim(0, 360)
ax[0].set_xlabel("Carrington Longitude [Deg.]")
ax[0].set_ylabel("Carrington Latitude [Deg.]")
ax[0].set_title("Photosphere")
plt.tight_layout()

pos = ax[1].imshow(pfss_out.source_surface_br.data, extent=[0,360,-90, 90], origin="lower", aspect="auto", cmap="seismic", vmin=-0.2, vmax=0.2)
cbar = fig.colorbar(pos, ax=ax[1], orientation="vertical")
cbar.ax.set_ylabel(r'$B_{r}(r=2.5R_{S})$ [Gauss]', rotation=90)
ax[1].set_xlabel("Carrington Longitude [Deg.]")
ax[1].set_title("Source Surface ")
plt.savefig("figs/CR" + str(CR) + "/pfss_result.png")

In [ ]:
tracer = tracing.FortranTracer()
r = 1.2 * const.R_sun
lat = np.linspace(-np.pi / 2, np.pi / 2, 15)
lon = np.linspace(0, 2 * np.pi, 15)
lat, lon = np.meshgrid(lat, lon, indexing='ij')
lat, lon = lat.ravel() * u.rad, lon.ravel() * u.rad

seeds = SkyCoord(lon, lat, r, frame=pfss_out.coordinate_frame)
field_lines = tracer.trace(seeds, pfss_out)

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(7, 5))
ax = fig.add_subplot(111, projection='3d')

for field_line in field_lines:
    color = {0: 'black', -1: 'tab:blue', 1: 'tab:red'}.get(field_line.polarity)
    coords = field_line.coords
    coords.representation_type = 'cartesian'
    ax.plot(coords.x / const.R_sun,
            coords.y / const.R_sun,
            coords.z / const.R_sun,
            color=color, linewidth=0.9)
ax.set_xlabel(r"Carrington X [$R_{S}$]")
ax.set_ylabel(r"Carrington Y [$R_{S}$]")
ax.set_zlabel(r"Carrington Z [$R_{S}$]")
plt.savefig("figs/CR" + str(CR) + "/PFSS_field_lines_spherical_CR"+ str(CR) + ".png", dpi=600)

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(7, 5))
ax = fig.add_subplot(111, projection='3d')

for field_line in field_lines:
    color = {0: 'black', -1: 'tab:blue', 1: 'tab:red'}.get(field_line.polarity)
    coords = field_line.coords
    coords.representation_type = 'spherical'
    ax.scatter(
            coords.lon.value,
            coords.lat.value,
            coords.radius / const.R_sun,
            color=color, s=0.1)

ax.set_zlim(1, 2.5)
ax.set_xticks([0, 90, 180, 270, 360])
ax.set_yticks([-90, -45, 0, 45, 90])
ax.set_xlabel("Longitude [Deg.]")
ax.set_ylabel("Latitude [Deg.]")
ax.set_zlabel(r"Radial Distance [$R_{S}$]")
plt.tight_layout()
plt.savefig("figs/CR" + str(CR) + "/PFSS_field_lines_cartesian.png", dpi=600)

# Magnetic Expansion Factor

In [ ]:
# trace the magnetic field lines for the ACE projection to obtain the expansion factor
tracer = tracing.FortranTracer()
seeds = SkyCoord(earth_coords.lon.to(u.rad), earth_coords.lat.to(u.rad), 2.5 * const.R_sun, frame=pfss_out.coordinate_frame)
field_lines_fp = tracer.trace(seeds, pfss_out)
fp_ace_traj = field_lines_fp.expansion_factors

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
_ = ax.scatter(earth_coords.lon, np.log(fp_ace_traj), s=1)
_ = ax.set_xticks([0, 55, 112, 167, 223, 278, 333, 360])
_ = ax.set_xlabel("Carrington Longtiude [Deg.]")
_ = ax.set_title("PFSS Magnetic Expansion Factor \n at ACE's Projected Trajectory")
plt.gca().invert_xaxis()
_ = ax.set_xlim(360, 0)
_ = ax.set_ylabel("$\ln(f_{p})$")
plt.tight_layout()
plt.savefig("figs/CR" + str(CR) + "/magentic_expansion_factor.png", dpi=800)

# Coronal Hole Mapping

In [ ]:
topologies = pfss2flines(pfss_out)
expansion_factor = pfss2flines(pfss_out, trace_from_SS=True)

In [ ]:
lons = np.linspace(0,360,361)
lats = np.linspace(-90,90,181)
fig,ax = plt.subplots(ncols =2, sharex=True, sharey=True, figsize=(10,3.5))
pos = ax[0].pcolormesh(lons,lats,topologies,cmap="coolwarm")
cbar = fig.colorbar(pos, ax=ax[0], orientation="vertical")
cbar.ax.set_ylabel(r'', rotation=90)
pos = ax[1].pcolormesh(lons,lats,np.log(expansion_factor))
cbar = fig.colorbar(pos, ax=ax[1], orientation="vertical")
cbar.ax.set_ylabel(r'$\ln(f_{p})$ [radians]', rotation=90)
_ = ax[0].set_xlabel("Carrington Longitude [deg.]")
_ = ax[1].set_xlabel("Carrington Longitude [deg.]")
_ = ax[0].set_ylabel("Latitude [deg.]")
_ = ax[0].set_xticks(np.linspace(0,360,5))
_ = ax[0].set_yticks(np.linspace(-90,90,5))
_ = ax[0].set_title(r"Coronal Hole Map")
_ = ax[1].set_title(r"Magnetic Expansion Factor")

for ii in range(7, len(ACE_longitude.to(u.rad))-5):
    if ii % 15==0:
        field_line = field_lines_fp[ii]
        coords = field_line.coords
        color= {0: 'black', -1: 'blue', 1: 'red'}.get(field_line.polarity)
        ax[0].scatter(coords.lon, coords.lat, c=color, s=0.05)

plt.tight_layout()
plt.savefig("figs/CR" + str(CR) + "/coronal_hole_plot_CR" + str(CR) + ".png", dpi=600)

### Great-circle angular distance from the nearest coronal hole boundary at the photosphere $d$

$d = \arccos\left[\cos(\theta_{1}) \cos(\theta_{2}) \cos(\phi_{1}- \phi_{2}) + \sin(\theta_{1}) \sin(\theta_{2})\right]$

In [ ]:
d = distance_to_coronal_hole_boundary(topologies, field_lines_fp)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
#_ = ax.plot(p_interp, d_ace_traj, label="interpolated")
_ = ax.scatter(earth_coords.lon, d, s=2, c="k")

_ = ax.set_xticks([0, 50, 100, 150,  200, 250, 300, 360])
plt.gca().invert_xaxis()
_ = ax.set_xlim(360, 0)
_ = ax.set_ylim(0, 0.09)
_ = ax.set_xlabel("Carrington Longtiude [Deg.]")
_ = ax.set_title("Distance to Coronal Hole Boundary\n at ACE's Projected Trajectory")
_ = ax.set_ylabel("$d [radians]$")
#_ = ax.legend()
plt.tight_layout()
plt.savefig("figs/CR" + str(CR) + "/distance_to_coronal_hole_boundary_CR" + str(CR) + ".png", dpi=600)

# WSA

WSA is an empirical model of the ambient solar wind. The WSA model is based on the inverse relationship between the solar wind speed and the magnetic field expansion
factor $f_{p}$ and the minimum angular distance that an open field
footpoint lies from the nearest coronal hole boundary $d$, such that
\begin{equation}
    f_{p} = \left(\frac{1}{r_{\text{SS}}}\right)^2\frac{B_{r}(r_{0}, \theta_{0}, \phi_{0})}{B_{r}(r_{SS}, \theta_{1}, \phi_{1})}
\end{equation}
where the indicies $0$ and $1$ refer to the longitude and latitude at the solar surface and upper boundary. These values are obtained by tracing the field lines down from $r_{SS}$ using a 4th order Runge Kutta scheme. The WSA model relation is given by 
\begin{equation}
    v_{\text{wsa}}(f_{p}, d) = v_{0} + \frac{v_{1} - v_{0}}{(1+f_{p})^{\alpha}} \left(\beta - \gamma \exp \left(-\left(\frac{d}{\omega}\right)^{\delta} \right)\right)^{\psi}
\end{equation}
The authors in [(Kumar et al. 2020)](https://www.frontiersin.org/articles/10.3389/fspas.2020.572084/full) set $v_{0} = 250 \frac{\text{km}}{\text{s}}$, $v_{1} = 900 \frac{\text{km}}{\text{s}}$, $\alpha = 1.5/9$, $\beta = 1$, $\gamma = 0.8$, $\omega = 0.03$, $\delta = 1.5$, $\psi=3$.


The authors in [(Bailey et al. 2021)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2020SW002673) set $v_{0} = 250 \frac{\text{km}}{\text{s}}$, $v_{1} = 650 \frac{\text{km}}{\text{s}}$, $\alpha = 0.19$, $\beta = 1$, $\gamma = 0.8$, $\omega = 3^{\circ}$, $\delta = 1.75$, $\psi = 3$.


The authors in [(Reiss et al. 2016)](https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1002/2016SW001390) set $v_{0} = 250 \frac{\text{km}}{\text{s}}$, $v_{1} = 1125 \frac{\text{km}}{\text{s}}$, $\alpha = 0.2$, $\beta = 1$, $\gamma = 0.8$, $\omega = 2.6^{\circ}$, $\delta = 1.25$, $\psi = 2.5$.


In [ ]:
v0 = 250
v1 = 850
alpha= 0.4
beta = 1.25
gamma = 0.2
w = 0.01 # to radians
delta = 1.75
psi=3

# dchb
omega = 0.05
epsilon = 0.01
v0_dchb = 250
v1_dchb = 750

In [ ]:
v_wsa = v0 + (v1 - v0)/((1+fp_ace_traj)**alpha) * (beta - gamma* np.exp(-(d/w)**delta))**psi
v_ws = v0 + (v1 - v0)/(fp_ace_traj)**alpha
v_dchb = v0 + 0.5*(v1_dchb - v0_dchb) * (1 + np.tanh((d - epsilon)/omega))

In [ ]:
p_hux = np.linspace(0, 2*np.pi, len(earth_coords.lon))

v_wsa_interp = interpolate_ace_data(x=p_hux, xp=earth_coords.lon.to(u.rad).value, fp=v_wsa, period=2*np.pi)
v_ws_interp = interpolate_ace_data(x=p_hux, xp=earth_coords.lon.to(u.rad).value, fp=v_ws, period=2*np.pi)
v_dchb_interp = interpolate_ace_data(x=p_hux, xp=earth_coords.lon.to(u.rad).value, fp=v_dchb, period=2*np.pi)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
_ = ax.scatter(earth_coords.lon, v_wsa, c="orange", label="WSA", s=1)
_ = ax.scatter(earth_coords.lon, v_ws,  label="WS",c="green", s=1)
_ = ax.scatter(earth_coords.lon, v_dchb,label="DCHB", c="blue",s=1)
_ = ax.plot(p_hux*180/np.pi, v_wsa_interp, c="orange",label="WSA")
_ = ax.plot(p_hux*180/np.pi, v_ws_interp, c="green", label="WS")
_ = ax.plot(p_hux*180/np.pi, v_dchb_interp, c="blue",label="DCHB")
_ = ax.set_xticks([0, 90, 180, 270, 360])
_ = ax.set_xlim(0, 360)
#_ = ax.set_ylim(0, 900)
_ = ax.set_xlabel("Carrington Longitude [Deg.]")
_ = ax.set_ylabel("Solar Wind Velocity [km/s]")
_ = ax.set_title(r"WS/WSA/DCHB results at $r_{SS}$")
ax.legend()

In [ ]:
np.cos(np.mean(np.pi/2 - earth_coords.lat.to(u.rad).value))**2

In [ ]:
r_vec = (np.linspace(2.5, np.max(ACE_r.to(u.solRad)).value, 500) * u.solRad).to(u.km).value
dr_vec = r_vec[1:] - r_vec[:-1]
dp_vec = p_hux[1:] - p_hux[:-1]

vr_hux_wsa = apply_hux_f_model(initial_condition=v_wsa_interp, 
                               dr_vec=dr_vec, 
                               dp_vec=dp_vec, 
                               alpha=0.15,
                               r0=2.5, 
                               rh=50, 
                               theta=np.mean(np.pi/2 - earth_coords.lat.to(u.rad).value))
vr_hux_ws = apply_hux_f_model(initial_condition=v_ws_interp, 
                              dr_vec=dr_vec, 
                              dp_vec=dp_vec, 
                              alpha=0.15, 
                              r0=2.5, 
                              rh=50, 
                              theta=np.mean(np.pi/2 - earth_coords.lat.to(u.rad).value))
vr_hux_dchb = apply_hux_f_model(initial_condition=v_dchb_interp, 
                                dr_vec=dr_vec, 
                                dp_vec=dp_vec, 
                                alpha=0.15,
                                r0=2.5, 
                                rh=50, 
                                theta=np.mean(np.pi/2 - earth_coords.lat.to(u.rad).value))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))
ax.plot(p_hux*180/np.pi, vr_hux_wsa[-1, :], "b", label=r"$PFSS \rightarrow WSA \rightarrow HUX$")
ax.plot(p_hux*180/np.pi, vr_hux_ws[-1, :], "r", label=r"$PFSS \rightarrow WS \rightarrow HUX$")
ax.plot(p_hux*180/np.pi, vr_hux_dchb[-1, :], "orange", label=r"$PFSS \rightarrow DCHB \rightarrow HUX$")
ax.scatter(ACE_longitude,  ACE_vr, s=1, c="k", label="ACE Observations at L1")
ax.set_title("CR "+ str(CR))
ax.set_xticks([0, 90, 180, 270, 360])
ax.set_xlim(0, 360)
ax.set_xlabel("Carrington Longitude [Deg.]")
ax.set_ylabel("Solar Wind \n Radial Velocity [km/s]")
ax.legend()

In [ ]:
vr_wsa_interp = interp_2d_ace_hux(p_hux, r_vec, vr_hux_wsa, ACE_r, ACE_longitude)
vr_ws_interp = interp_2d_ace_hux(p_hux, r_vec, vr_hux_ws, ACE_r, ACE_longitude)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(ACE_obstime.value, vr_wsa_interp, "b", label=r"$PFSS \rightarrow WSA \rightarrow HUX$")
ax.plot(ACE_obstime.value,  ACE_vr, c="k", ls="--", label="ACE Observations at L1")
ax.set_title("CR " + str(CR))
ax.set_ylabel("Solar Wind Radial Velocity [km/s]")
ax.legend()
ax.set_ylim(250, 900)

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=100))
plt.savefig("figs/CR" + str(CR) + "/ACE_radial_velocity_CR" + str(CR) + ".png", dpi=600)